In [ ]:
# 本脚本用于根据一定的策略计算可交易日期
from opentrader.core.ticker import *
from opentrader.common.db import db_ot
from opentrader.common.utils import d2dt
from opentrader.agents.ths.api import symbol_convert3
from pymongo.errors import *
from datetime import datetime, timedelta
import pymongo

start_date = gen_time('2015-01-01 00:00:00').date()
end_date = gen_time('2015-12-17 00:00:00').date()
date = start_date
dates = []
while date <= end_date:
    if TradeCalendar.check_date(date):
        dates.append(date)
    date += timedelta(1)

trade_dates = []
recent_highs = []
recent_high_num = 20
recent_lows = []
recent_low_num = 10
result = db_ot.xueqiu_k_day.find({"symbol":"SH000001"}).sort('time', pymongo.ASCENDING)
status = True # True for buy, False for empty

"""
for each in result:
    if status == True:
        if len(recent_lows) == 0 or each['close'] < min(recent_lows):
            status = False
        else:
            trade_dates.append(each['time'].date())
    else:
        if len(recent_highs) == 0 or each['close'] > max(recent_highs):
            status = True
            trade_dates.append(each['time'].date())
    
    # update recent highs and lows
    recent_highs.append(each['high'])
    if len(recent_highs) > recent_high_num:
        recent_highs.pop(0)
    
    recent_lows.append(each['low'])
    if len(recent_lows) > recent_low_num:
        recent_lows.pop(0)
"""
for each in result:
    if status == True:
        if each['close'] < each['ma20']:
            status = False
        else:
            trade_dates.append(each['time'].date())
    else:
        if each['close'] > each['ma20']:
            status = True
            trade_dates.append(each['time'].date())


# 统计各年交易天数
year_days = {}
for each in trade_dates:
    if each.year not in year_days:
        year_days[each.year] = 1
    else:
        year_days[each.year] += 1
print(year_days)

trade_dates_json = {}
for each in trade_dates:
    trade_dates_json[str(each)] = 1
    
import json
f = open('iwencai_flag_trade_dates.json', 'w')
f.write(json.dumps(trade_dates_json))
f.close()